In [2]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import datetime as DT
import io

In [3]:
purchase_data = pd.read_excel('./Purchase Data/Combined Purchase Data_Final.xlsx')
survey_data = pd.read_excel('./Survey Data/Combined_Response_Data.xlsx')

# 1) Data Cleaning

    1) Most of the Data Cleaning was Done Manually
    2) Group 1 and Group 2 data was pretty much botched up
    3) GST and MRP data was botched up for majority
    4) Order Date data was also not correct, as Group 1 and Group 2 were using their own version of date formatting
    5) State and City Data was also botched up.
    5) Categories, Subcategories and Ratings was added using Beautifull soup from FLipkart website directly.
    6) Not every product was found, we had to manually add data for those products.
    7) Above all aspects of Data Cleaning was done manually
    8) Any remaining Data cleaning will be done in pythonic way

## 1.1) Purchase Data

###### Checking %ntage of NaN data before Cleaning

In [52]:
print('% Data NaN\n',purchase_data.isna().sum()*100/purchase_data.shape[0])


% Data NaN
 Invoice ID         0.000000
Name on Invoice    0.000000
Order Date         0.000000
State              0.000000
City               0.000000
Pin                0.000000
Title              0.000000
Categories         0.000000
Subcategories      0.000000
Ratings            2.044668
Quantity           0.000000
MRP                0.000000
Discount           0.000000
Delivery Fee       0.000000
GST                0.000000
Final Price        0.000000
GST%               0.000000
dtype: float64


###### Mean Imputation for Ratings Column

In [53]:
purchase_data.loc[purchase_data['Ratings'].isna(),'Ratings'] = purchase_data['Ratings'].mean()

###### Checking %ntage of NaN data after Cleaning / Imputation

In [54]:
print('% Data NaN\n',purchase_data.isna().sum()*100/purchase_data.shape[0])


% Data NaN
 Invoice ID         0.0
Name on Invoice    0.0
Order Date         0.0
State              0.0
City               0.0
Pin                0.0
Title              0.0
Categories         0.0
Subcategories      0.0
Ratings            0.0
Quantity           0.0
MRP                0.0
Discount           0.0
Delivery Fee       0.0
GST                0.0
Final Price        0.0
GST%               0.0
dtype: float64


###### Converting Column to Date type

In [55]:
purchase_data['Order Date'] =  pd.to_datetime(purchase_data['Order Date'])

###### Converting Numeric GST Value to Proper Slabs

In [56]:
purchase_data.loc[purchase_data['GST%']<=0.025,'GST%%'] = '0%'
purchase_data.loc[(purchase_data['GST%']>0.025) & (purchase_data['GST%']<=0.08) ,'GST%%'] = '5%'
purchase_data.loc[(purchase_data['GST%']>0.08) & (purchase_data['GST%']<=0.145),'GST%%'] = '12%'
purchase_data.loc[(purchase_data['GST%']>0.145) & (purchase_data['GST%']<=0.23),'GST%%'] = '18%'
purchase_data.loc[(purchase_data['GST%']>0.23),'GST%%'] = '28%'

purchase_data.drop('GST%',axis=1,inplace=True)

In [57]:
purchase_data.to_csv('Cleaned_Combined_Purchase_data.csv')

## 1.2) Survey Data

 ###### Checking %ntage of NaN data before Cleaning


In [72]:
print('% Data NaN\n',survey_data.isna().sum()*100/survey_data.shape[0])


% Data NaN
 Name                                                                                    0.000000
Gender                                                                                  0.000000
Date of birth                                                                           0.000000
Current Job Title                                                                       0.000000
Current City of Residence                                                               0.000000
Language spoken at home/Mother tongue                                                   0.000000
Marital Status                                                                          0.000000
Number of children                                                                      0.000000
Do you have to care for anyone with chronic illness?                                    0.000000
Income (per month)                                                                      0.000000
Preferred Mode of 

###### Mode Imputation for categorical Columns

In [73]:
survey_data.loc[survey_data['Please list your Hobbies/Interests'].isna(),'Please list your Hobbies/Interests'] \
= survey_data['Please list your Hobbies/Interests'].mode()[0]

survey_data.loc[survey_data['How often do you pursue your hobbies?'].isna(),'How often do you pursue your hobbies?'] \
= survey_data['How often do you pursue your hobbies?'].mode()[0]


###### Median Imputation 

In [74]:
columns_to_fix = ['In most ways, my life is close to my ideal',
       'The Conditions in my life are excellent.',
       'So far, I have got the important things I want in my life',
       'If I could live my life over, I would change almost nothing',
       'Many people have directly or indirectly contributed to my progress in life',
       'I feel good when I co-operate with others',
       'When making a decision, I take other people\'s needs and feelings into account.',
       'It is my duty to take care of my family, even when I have to sacrifice what I want.',
       'Honesty is important to achieve success',
       'Social inequalities bother me',
       'What I am today is solely because of my hard work and talent',
       'People should keep their troubles to themselves',
       'It is important that I do my job better than others',
       'Winning is everything',
       'I don’t worry about others as long as I am happy',
       'It upsets me when my work is not recognized by others',
       'Traditional values are important for me', 'I plan for the long-term',
       'I am willing to give up today’s fun for future success',
       'I believe persistence is key to success',
       'Saving money is important to me',
       'It is okay to use shortcuts to get what you want',
       'I like to get quick results',
       'I would rather spend money today than save for future',
       'When my routine is disturbed, it upsets me',
       'I admire people who own expensive homes, cars, and clothes',
       'The things I own say a lot about how well I’m doing in life',
       'I aspire a luxurious and comfortable lifestyle',
       'My life would be better if I owned certain things I don’t have',
       'I’d be happier if I could afford to buy more things',
       'I don’t pay much attention to the material objects other people own',
       'I usually buy only the things I need',
       'I have all the things I really need to enjoy life',
       'My happiness does not depend on things I own',
       'There is a higher purpose to life than comfort and luxury',
       'I believe success in life does not mean becoming rich',
       'I segregate waste before its disposal', 'I try to conserve water',
       'I try to educate people I know about climate change',
       'I sign petitions related to environmental issues',
       'I try to conserve electricity',
       'I walk/cycle/use public transport to save fuel']

In [75]:
for i in columns_to_fix:
    survey_data.loc[survey_data[i].isna(),i] \
    = survey_data[i].median()

 ###### Checking %ntage of NaN data after Cleaning / Imputation


In [76]:
print('% Data NaN\n',survey_data.isna().sum()*100/survey_data.shape[0])


% Data NaN
 Name                                                                                   0.0
Gender                                                                                 0.0
Date of birth                                                                          0.0
Current Job Title                                                                      0.0
Current City of Residence                                                              0.0
Language spoken at home/Mother tongue                                                  0.0
Marital Status                                                                         0.0
Number of children                                                                     0.0
Do you have to care for anyone with chronic illness?                                   0.0
Income (per month)                                                                     0.0
Preferred Mode of Payment                                                     

In [77]:
survey_data.to_csv('Cleaned_Survey_data.csv')

###### Checkpoint 1

In [75]:
survey_data_final = pd.read_csv('Cleaned_Survey_data.csv')
purchase_data_final = pd.read_csv('Cleaned_Combined_Purchase_data.csv')

purchase_data_final.drop('Unnamed: 0',axis=1,inplace=True)
survey_data_final.drop('Unnamed: 0',axis=1,inplace=True)

##Convert to Date Format
purchase_data_final['Order Date'] = pd.to_datetime(purchase_data_final['Order Date'])
survey_data_final['Date of birth'] = pd.to_datetime(survey_data_final['Date of birth'])
#purchase_data_final['Order Date'] = purchase_data_final['Order Date'].dt.strftime('%d/%m/%Y')
purchase_data_final['Order Date'] = pd.to_datetime(purchase_data_final['Order Date'])


# 2) Feature Engineering

## 2.1) Purchase Data

###### Sale

In [2]:

purchase_data_final = pd.read_csv('Cleaned_Combined_Purchase_data_Final_Feature_Engineering_2.csv')

In [79]:
purchase_data_final.groupby('Sale').count()

,Invoice ID,Name on Invoice,Order Date,State,City,Pin,Title,Categories,Subcategories,Ratings,Quantity,MRP,Discount,Delivery Fee,GST,Final Price,GST%%
Sale,,,,,,,,,,,,,,,,,
No,2468,2468,2468,2468,2468,2468,2468,2468,2468,2468,2468,2468,2468,2468,2468,2468,2468
Yes,711,711,711,711,711,711,711,711,711,711,711,711,711,711,711,711,711


In [77]:
def sale_or_not(df,start_date,end_date):
    df.loc[(df['Order Date']>start_date)  & (df['Order Date']<=end_date),'Sale'] = 'Yes'
    df['Order Date'] = pd.to_datetime(df['Order Date'])
    return df

purchase_data_final['Sale'] = 'No'

purchase_data_final = sale_or_not(purchase_data_final,'2019-01-20','2019-01-22')
purchase_data_final = sale_or_not(purchase_data_final,'2019-06-01','2019-06-03')
purchase_data_final = sale_or_not(purchase_data_final,'2019-03-07','2019-06-08')
purchase_data_final = sale_or_not(purchase_data_final,'2019-05-15','2019-05-19')
purchase_data_final = sale_or_not(purchase_data_final,'2019-06-11','2019-06-15')
purchase_data_final = sale_or_not(purchase_data_final,'2019-07-05','2019-07-18')
purchase_data_final = sale_or_not(purchase_data_final,'2019-08-08','2019-08-10')
purchase_data_final = sale_or_not(purchase_data_final,'2019-09-29','2019-10-04')
purchase_data_final = sale_or_not(purchase_data_final,'2019-10-12','2019-10-16')
purchase_data_final = sale_or_not(purchase_data_final,'2019-12-01','2019-12-05')
purchase_data_final = sale_or_not(purchase_data_final,'2019-12-21','2019-12-23')

purchase_data_final = sale_or_not(purchase_data_final,'2020-01-19','2020-01-22')
purchase_data_final = sale_or_not(purchase_data_final,'2020-03-19','2020-03-22')
purchase_data_final = sale_or_not(purchase_data_final,'2020-06-23','2020-06-27')
purchase_data_final = sale_or_not(purchase_data_final,'2020-07-08','2020-07-12')
purchase_data_final = sale_or_not(purchase_data_final,'2020-08-06','2020-08-10')
purchase_data_final = sale_or_not(purchase_data_final,'2020-09-18','2020-09-20')
purchase_data_final = sale_or_not(purchase_data_final,'2020-10-16','2020-10-21')
purchase_data_final = sale_or_not(purchase_data_final,'2020-10-29','2020-11-04')
purchase_data_final = sale_or_not(purchase_data_final,'2020-11-08','2020-11-13')

purchase_data_final = sale_or_not(purchase_data_final,'2021-01-01','2021-01-03')
purchase_data_final = sale_or_not(purchase_data_final,'2021-01-20','2021-02-24')
purchase_data_final = sale_or_not(purchase_data_final,'2021-02-01','2021-02-03')
purchase_data_final = sale_or_not(purchase_data_final,'2021-03-24','2021-03-26')
purchase_data_final = sale_or_not(purchase_data_final,'2021-04-01','2021-04-03')
purchase_data_final = sale_or_not(purchase_data_final,'2021-04-09','2021-04-14')
purchase_data_final = sale_or_not(purchase_data_final,'2021-05-02','2021-05-07')
purchase_data_final = sale_or_not(purchase_data_final,'2021-06-13','2021-06-16')
purchase_data_final = sale_or_not(purchase_data_final,'2021-06-23','2021-06-28')
purchase_data_final = sale_or_not(purchase_data_final,'2021-07-25','2021-07-29')
purchase_data_final = sale_or_not(purchase_data_final,'2021-07-01','2021-07-03')
purchase_data_final = sale_or_not(purchase_data_final,'2021-08-10','2021-08-12')
purchase_data_final = sale_or_not(purchase_data_final,'2021-08-18','2021-08-20')
purchase_data_final = sale_or_not(purchase_data_final,'2021-08-05','2021-08-09')
purchase_data_final = sale_or_not(purchase_data_final,'2021-09-07','2021-09-11')


###### Covid

In [81]:
purchase_data_final

,Invoice ID,Name on Invoice,Order Date,State,City,Pin,Title,Categories,Subcategories,Ratings,Quantity,MRP,Discount,Delivery Fee,GST,Final Price,GST%%,Sale,Covid
0,OD103719706054443200,G3M1R1,2015-08-23,Delhi,New Delhi,110063,Nova 2 SLICE SANDWICH GRILL MAKER Black &,Home & Kitchen,Kitchen Appliances,4.100000,1,1370.33,0.0,174.67,195.66,1545.00,18%,No,No
1,OD106420064045076000,G3M1R1,2016-01-07,Delhi,New Delhi,110031,Omron HEM-7120 Bp MonitoR,Health & Personal Care Appliances,Health Care,4.300000,1,1164.76,0.0,95.24,60.00,1260.00,5%,No,No
2,OD106420064045076001,G3M1R1,2016-01-07,Delhi,New Delhi,110032,Johnson & Johnson One Touch Select Glucose Mon...,Clothing and Accessories,Books,4.500000,1,596.67,0.0,133.33,34.77,730.00,5%,No,No
3,OD109711988579254000,G3M1R2,2017-07-17,Delhi,New Delhi,110003,"HAANS Shakeit 500 ml Shaker, Sipper",Exercise & Fitness,Fitness Accessories,4.300000,1,199.00,0.0,0.00,30.36,199.00,18%,No,No
4,OD109711988579254000,G3M1R2,2017-07-18,Delhi,New Delhi,110003,Muscletech Premium 100% Whey Protein Chocolate,Health Care,Health Supplements,4.100000,1,4162.00,0.0,0.00,910.44,4162.00,28%,No,No
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3174,FAC5OU2100002091,G1M2R12,2020-05-18,Kerala,Anakkayam,676509,Patanjali DIVYA MADHUNASHINI VATI EXTRA,Health Care,Home Medicines,4.200000,1,563.00,0.0,59.00,0.00,622.00,0%,No,No
3175,FADVJD2100007635,G1M2R12,2020-10-18,Kerala,Anakkayam,676509,DIVYA PHARMACY KAISHORE GUGGUL 80,Health Care,Home Medicines,3.900000,1,234.00,0.0,48.21,28.08,282.21,12%,Yes,1st Wave
3176,OD117537329774872000,G1M1R5,2020-01-09,Uttar Pradesh,Prayagraj,211002,Cables Kart Jumbo Extended Waterproof Foldable...,Computers,Computer Peripherals,4.118337,1,358.00,0.0,59.00,45.61,417.00,18%,No,No
3177,OD120189180140808000,G1M1R5,2020-11-18,Uttar Pradesh,Prayagraj,211002,Realme Larger TV Installation and Demo,Home Entertainment,Televisions,4.300000,1,500.00,0.0,0.00,0.00,500.00,0%,No,1st Wave


In [83]:
purchase_data_final.to_csv('dump.csv')

In [82]:
def covid_or_not(df,start_date,end_date,wave):
    df.loc[(df['Order Date']>start_date)  & (df['Order Date']<=end_date),'Covid'] = wave
    df['Order Date'] = pd.to_datetime(df['Order Date'])
    return df
purchase_data_final['Covid'] = 'No'
purchase_data_final = covid_or_not(purchase_data_final,'2020-04-01','2020-10-30','1st Wave')
purchase_data_final = covid_or_not(purchase_data_final,'2021-03-20','2021-06-03','2nd Wave')



###### Tiers

In [7]:
tier_1 = ['Delhi','Pune','Ahmedabad','Bengaluru','Chennai','Hyderabad','Kolkata','Mumbai']
tier_2 = ['Agra','Ajmer','Aligarh','Amravati','Amritsar','Asansol','Aurangabad','Bareilly','Belgaum','Bhavnagar','Bhiwandi','Bhopal','Bhubaneswar',
'Bikaner','Bilaspur','Bokaro Steel City','Chandigarh','Coimbatore','Cuttack','Dehradun','Dhanbad','Bhilai','Durgapur','Erode','Faridabad',
'Firozabad','Ghaziabad','Gorakhpur','Gulbarga','Guntur','Gwalior','Gurugram','Guwahati','Hamirpur','Hubli–Dharwad','Indore','Jabalpur',
'Jaipur','Jalandhar','Jalgaon','Jammu','Jamnagar','Jamshedpur','Jhansi','Jodhpur','Navi Mumbai ','Kakinada','Kannur','Kanpur','Karnal',
'Kochi','Kolhapur','Kollam','Kozhikode','Kurnool','Ludhiana','Lucknow','Madurai','Malappuram','Mathura','Mangalore','Meerut','Moradabad',
'Mysore','Nagpur','Nanded','Nashik','Nellore','Navi Mumbai','Noida','Patna','Puducherry','Purulia','Prayagraj','Raipur','Rajkot','Rajamahendravaram',
'Ranchi','Rourkela','Ratlam','Salem','Sangli','Shimla','Siliguri','Solapur','Srinagar','Surat','Thanjavur','Thiruvananthapuram','Thrissur',
'Tiruchirappalli','Tirunelveli','Tiruvannamalai','Ujjain','Vijayapura','Vadodara','Varanasi','Vasai-Virar City','Vijayawada','Visakhapatnam','Vellore and Warangal']

In [128]:
purchase_data_final['City_Tier'] = 'Tier 3'

for i in tier_1:
    purchase_data_final.loc[purchase_data_final['State']==i,'City_Tier'] = 'Tier_1'
for i in tier_1:
    purchase_data_final.loc[purchase_data_final['City']==i,'City_Tier'] = 'Tier_1'
    
    
for i in tier_2:
    purchase_data_final.loc[purchase_data_final['City']==i,'City_Tier'] = 'Tier_2'

###### Discount %

In [136]:
purchase_data_final['Discount%'] = np.ceil(100*purchase_data_final['Discount']/purchase_data_final['MRP'])

###### Delievery Fee%

In [138]:
purchase_data_final['Delivery Fee%'] = np.ceil(100*purchase_data_final['Delivery Fee']/purchase_data_final['MRP'])

###### Checkpoint 2

In [146]:
purchase_data_final.drop(['GST','Discount','Unnamed: 0','Pin','Title ','Delivery Fee'],axis=1,inplace=True)

purchase_data_final.to_csv('Cleaned_Combined_Purchase_data_Final_Feature_Engineering_2.csv')


## 2.2) Survey Data

In [2]:
survey_data_final = pd.read_csv('Cleaned_Survey_data.csv')

###### Age

In [3]:
pd.options.mode.chained_assignment = 'warn'
now = pd.Timestamp('now')
survey_data_final['Date of birth'] = pd.to_datetime(survey_data_final['Date of birth'])
survey_data_final['Date of birth'] = pd.to_datetime(survey_data_final['Date of birth'], format='%m%d%y')
survey_data_final['Date of birth'] = survey_data_final['Date of birth'].where(survey_data_final['Date of birth'] < now, survey_data_final['Date of birth'] -  np.timedelta64(100, 'Y'))   # 2
survey_data_final['Age'] = (now - survey_data_final['Date of birth']).astype('<m8[Y]')    # 3

###### has_child

In [4]:
survey_data_final['has_child'] = 'No'
survey_data_final.loc[survey_data_final['Number of children']>0,'has_child'] = 'Yes'

In [8]:
survey_data_final['Lives_in_city_tier'] = 'Tier 3'


for i in tier_1:
    survey_data_final.loc[survey_data_final['Current City of Residence']==i,'Lives_in_city_tier'] = 'Tier_1'
    
    
for i in tier_2:
    survey_data_final.loc[survey_data_final['Current City of Residence']==i,'Lives_in_city_tier'] = 'Tier_2'

In [9]:
survey_data_final.drop(['Date of birth','Number of children','Unnamed: 0','Current City of Residence'],axis=1,inplace=True)


In [10]:
survey_data_final.to_csv('Cleaned_Survey_data_Final_Feature_Engineering_2.csv')
